# Протоколы аутентификации с нулевым разглашением

In [1]:
IS_DEBUG = 1

In [2]:
def trace(*args, **kwargs):
    """
    Отладочная трассировка
    """
    
    global IS_DEBUG
    if IS_DEBUG:
        print('[TRACE]', end=' ')
        print(*args, **kwargs)

---

In [3]:
# Генерация простых чисел
from Crypto.Util import number

# КГПСЧ
from Crypto.Random import random

# Понадобится в протоколе Шнорра
from Crypto.PublicKey import RSA
from Crypto.Signature import pss
from Crypto.Hash import SHA256

# Немного математики
import gmpy2

# Для генерации идентификаторов клиентов
import uuid

---

In [4]:
def randint(a, b) -> gmpy2.mpz:
    """
    Генерация случайного числа с помощью КГПСЧ с поддержкой gmpy2
    """
    
    native_a = int(a)
    native_b = int(b)
    
    return gmpy2.mpz(random.randint(native_a, native_b))

def getrandbits(n) -> gmpy2.mpz:
    """
    """
    
    native_n = int(n)
    
    return gmpy2.mpz(random.getrandbits(native_n))

----

## Протокол Фиата-Шамира
![Fiat-Shamir Protocol](./images/Fiat-Shamir_2.png)

In [5]:
class FiatShamirCA(object):
    PRIME_BITS = 1024
    
    def __init__(self):
        """
        Инициализация доверенного центра.
        Генерируются два простых числа и их произведение.
        """
        
        self._db = {}
        
        self._p = number.getPrime(FiatShamirCA.PRIME_BITS)
        self._q = number.getPrime(FiatShamirCA.PRIME_BITS)
        self._modulus = gmpy2.mul(self._p, self._q)
        
        trace('[FiatShamirCA]', f'Modulus = {self._modulus}')
        
        
    def register_client(self, client_id: uuid.UUID, client_public_key: gmpy2.mpz) -> None:
        """
        Регистрация клиента. На вход принимает идентификатор клиента и его открытый ключ.
        Ничего не возвращает.
        Выбрасывает исключение, если клиент уже существует.
        """
        
        trace('[FiatShamirCA]', f'''Attempting to register client {client_id} 
        with public key {client_public_key}''')
        
        if client_id in self._db:
            trace('[FiatShamirCA]', 'User already exists')
            raise ValueError(f'Client {client_id} is already registered')
            
        #
        # Теперь сохраняю открытый ключ
        #
        
        self._db[client_id] = client_public_key
        trace('[FiatShamirCA]', 'Client registered successfully')
        
        
    def get_public_key(self, client_id: uuid.UUID) -> gmpy2.mpz:
        """
        Получение открытого ключа клиента. Получает на вход идентификатор клиента.
        Возвращает открытый ключ.
        Выбрасывает исключение, если пользователь отсутствует.
        """
        
        return self._db[client_id]
       
        
    @property
    def modulus(self) -> gmpy2.mpz:
        """
        Возвращает модуль.
        """
        
        return self._modulus

In [6]:
class FiatShamirClient(object):
    def __init__(self, modulus: gmpy2.mpz, *,
                 fake_data: tuple[uuid.UUID, gmpy2.mpz] = None):
        """
        Инициализация клиента с выработкой ключевой пары и идентификатора.
        Последний параметр требуются для демонстрации неудачной аутентификации.
        """
        
        self._id = uuid.uuid4() if fake_data is None else fake_data[0]
        self._r = None
        
        self._private_key = FiatShamirClient._generate_coprime(modulus)
        self._public_key = gmpy2.powmod(self._private_key, 2, modulus) if fake_data is None else fake_data[1]
        
        trace('[FiatShamirClient]', f'Client {self._id}')
        trace('[FiatShamirClient]', f'Public key = {self._public_key}')
        
        
    @property
    def public_key(self) -> gmpy2.mpz:
        """
        Получение открытого ключа.
        """
        
        return self._public_key
    
    
    @property
    def identifier(self) -> uuid.UUID:
        """
        Получение идентификатора.
        """
        
        return self._id
    
    
    def get_x(self, modulus: gmpy2.mpz) -> gmpy2.mpz:
        """
        Получение случайного значения, вычисляемого по формуле:
                    x = r ** 2 mod n,
        где n - модуль, r - случайное целое число из отрезка [1, n - 1].
        Значение r сохраняется.
        """
        
        self._r = randint(1, modulus - 1)
        return gmpy2.powmod(self._r, 2, modulus)
    
    
    def get_y(self, e: gmpy2.mpz, modulus: gmpy2.mpz) -> gmpy2.mpz:
        """
        Получение числа, вычисляемого по формуле:
                    y = r * s ** e mod n,
        где n - модуль, s - закрытый ключ пользователя, r - сгенерированное 
        функцией get_x внутреннее значение, e - целое число из отрезка [0, 1],
        пришедшее от проверяющего.
        """
        
        product = gmpy2.mul(self._r, gmpy2.powmod(self._private_key, e, modulus))
        return gmpy2.t_mod(product, modulus)
        
        
    @staticmethod
    def _generate_coprime(modulus: gmpy2.mpz) -> gmpy2.mpz:
        """
        Генерация случайного числа из отрезка [1, n - 1], взаимно
        простого с n, где n - модуль.
        """
        
        #
        # Генерирую число до тех пор, пока оно не станет взаимно простым с модулем
        #
        
        result = randint(1, modulus - 1)
        while gmpy2.gcd(result, modulus) != 1:
            result = randint(1, modulus - 1)
            
        return result

In [7]:
class FiatShamirVerifier(object):
    def __init__(self, iterations: int):
        """
        Инициализация проверяющего. Задает коичество итераций проверки.
        """
        
        self._iterations = iterations
        
        
    def authenticate(self, client: FiatShamirClient, ca: FiatShamirCA) -> bool:
        """
        Аутентификация клиента. На вход принимает дескрипторы клиента и СА.
        Возвращает True в случае успешной аутентификации.
        """
        
        trace('[FiatShamirVerifier]', f'Attempt to authenticate client {client.identifier}')
        trace('[FiatShamirVerifier]', f'Number of iterations = {self._iterations}')
        
        is_successful = True
        
        for i in range(self._iterations):
            x = client.get_x(ca.modulus)
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] {x = }')
            
            e = randint(0, 1)
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] {e = }')
            
            y = client.get_y(e, ca.modulus)
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] {y = }')
            
            pk = ca.get_public_key(client.identifier)
            product = gmpy2.mul(x, gmpy2.powmod(pk, e, ca.modulus))
            
            successful_iteration = gmpy2.powmod(y, 2, ca.modulus) == gmpy2.t_mod(product, ca.modulus)
            is_successful &= successful_iteration
            
            trace('[FiatShamirVerifier]', f'[Iteration {i + 1}] success: {successful_iteration}')
            
            if not successful_iteration:
                break
            
        print(f'Authentication successful: {is_successful}')
        return is_successful

----

In [8]:
# Создаю доверенный центр
ca = FiatShamirCA()

[TRACE] [FiatShamirCA] Modulus = 12191856898017159798587546422455421752858633626282060021598583357398772527530598588893544689136959730039102936840885514674504800643312223539786248696215947432124373256535457936717829425156232192058992360906359940219322511178687347261047567845423638755800890643695843876906927169999275414170891664211791249032439935692306258291681947815982878224586253785177098515274262589790002718175274870886800310544615174070654275442039963775342307597840200915998553791356958590294322641602548491595792236504739204592555906090152734043251306489762393498510479341599801143615663263070910535654507015144313529397957781455101423669877


In [9]:
# Создаю двух пользователей:
# - Алису - валидного пользователя
# - Еву - пользователя, представляющегося Алисой. но не знающего закрытого ключа Алисы
alice = FiatShamirClient(ca.modulus)
eve = FiatShamirClient(ca.modulus, 
                       fake_data=(alice.identifier, alice.public_key))

[TRACE] [FiatShamirClient] Client 0968e3b3-4324-4c96-a2f7-be52d040093d
[TRACE] [FiatShamirClient] Public key = 7405253908054208965388501637992637547335803914694863741623171676225738943835136417538008930534882622162158949655028830537107112765116169317583621352080486393371021734887312513829677580607710893940556942470929769414346728011374376425442079972151260801384904284270518954024971906158480092921573994027139855668690332115663386299472175842974858401062916461305634619283148141928476682950980805344080894462205845024966380602998610671526942613097572036486391480780596844651655760246719590217213180829616620810044486823327697010886834036977429262556922640663820451496666608897094325660325792767479754161009519275670274555188
[TRACE] [FiatShamirClient] Client 0968e3b3-4324-4c96-a2f7-be52d040093d
[TRACE] [FiatShamirClient] Public key = 74052539080542089653885016379926375473358039146948637416231716762257389438351364175380089305348826221621589496550288305371071127651161693175836213520804863933710

In [10]:
# Регистрирую валидного пользователя
ca.register_client(alice.identifier, alice.public_key)

[TRACE] [FiatShamirCA] Attempting to register client 0968e3b3-4324-4c96-a2f7-be52d040093d 
        with public key 7405253908054208965388501637992637547335803914694863741623171676225738943835136417538008930534882622162158949655028830537107112765116169317583621352080486393371021734887312513829677580607710893940556942470929769414346728011374376425442079972151260801384904284270518954024971906158480092921573994027139855668690332115663386299472175842974858401062916461305634619283148141928476682950980805344080894462205845024966380602998610671526942613097572036486391480780596844651655760246719590217213180829616620810044486823327697010886834036977429262556922640663820451496666608897094325660325792767479754161009519275670274555188
[TRACE] [FiatShamirCA] Client registered successfully


In [11]:
# Для простоты будет 3 итерации
ITERATIONS = 3

# Создаю проверяющую сторону
verifier = FiatShamirVerifier(ITERATIONS)

In [12]:
# Произвожу аутентификацию валидного пользователя
verifier.authenticate(alice, ca)

[TRACE] [FiatShamirVerifier] Attempt to authenticate client 0968e3b3-4324-4c96-a2f7-be52d040093d
[TRACE] [FiatShamirVerifier] Number of iterations = 3
[TRACE] [FiatShamirVerifier] [Iteration 1] x = mpz(6999942897129435064298676987542234838458769073229646276932195672399792209464144594348952527503845394976111417618169137875386209255075610932064250957884598174560777573383399823891507329314903603639813910273671179322575461908392865786517159244924847708927738318285303309393634783855823206017481596168797355629550331387866275112945044382653086060090734120911000028472337449372243615764063663839358692617737258245939447766494783268734824748962685944004244843469358638663842141026915371491139849222687238889713841830433467950469717309352218358379836861632251084558058517523231173748118079985967990275731090202493935070510)
[TRACE] [FiatShamirVerifier] [Iteration 1] e = mpz(1)
[TRACE] [FiatShamirVerifier] [Iteration 1] y = mpz(993715280471988602246136451185951560435402201955685523014882085550092743715

True

In [13]:
# Произвожу аутентификацию невалидного пользователя
verifier.authenticate(eve, ca)

[TRACE] [FiatShamirVerifier] Attempt to authenticate client 0968e3b3-4324-4c96-a2f7-be52d040093d
[TRACE] [FiatShamirVerifier] Number of iterations = 3
[TRACE] [FiatShamirVerifier] [Iteration 1] x = mpz(683182873663945878380666989065191092518501673576967410238884162364831860499694866729189004369203810978947468569778652706444050745091924274057578480141401268228560327172880305515507777040568437438628585677314386297850772308547332432255760950145584529258898556264109033370356627610885250062372088275169272512881596496285732635231911801765115851807451501266545691916716940777013943155236875685036064619745285114763583701438528478837235789207813290932913045585685032488446966894018288102990926648750600003938094815841791833969498610261137057535526396730888613427186938806517224403483201406384947624079961470862355028960)
[TRACE] [FiatShamirVerifier] [Iteration 1] e = mpz(1)
[TRACE] [FiatShamirVerifier] [Iteration 1] y = mpz(8844339706935543301181967075598920534797993418198739207474827558175896703842

False

---

## Протокол Шнорра
![Schnorr Protocol](./images/Schnorr_2.png)

In [14]:
class SchnorrCA(object):
    RSA_KEY_LENGTH = 4096
    
    def __init__(self):
        """
        Инициализация доверенного центра
        Генерация простых чисел p, q и элемента g, а также ключевой пары СА
        """
        
        self._db = {}
        
        #
        # Генерация параметров протокола
        #
        
        self._p, self._q, self._g = SchnorrCA._generate_parameters()
        
        #
        # Ключевая пара RSA для СА
        #
        
        self._private_key = RSA.generate(SchnorrCA.RSA_KEY_LENGTH)
        self._public_key = self._private_key.publickey()
        
        trace('[SchnorrCA]', f'RSA public key = {(self._public_key.e, self._public_key.n)}')
        
        
    def register_client(self, 
                        client_id: uuid.UUID, 
                        client_public_key: gmpy2.mpz) -> tuple[uuid.UUID, gmpy2.mpz, bytes]:
        """
        Регистрация клиента. Принимает на вход идентификатор клиента и его открытый ключ.
        Возвращает подписанный сервером сертификат.
        Выбрасывает исключение, если клиент уже зарегистрирован.
        """
        
        if client_id in self._db:
            trace('[SchnorrCA]', 'User already exists')
            raise ValueError(f'Client {client_id} is already registered')
            
        #
        # Создаю сертификат
        #
            
        h = SHA256.new(str(client_id).encode() + number.long_to_bytes(int(client_public_key)))
        signature = pss.new(self._private_key).sign(h)
        
        certificate = (client_id, client_public_key, signature)
        trace('[SchnorrCA]', f'Certificate = {certificate}')
        
        #
        # Сохраняю его и возвращаю
        #
        
        self._db[client_id] = certificate
        return certificate
    
    
    def verify_certificate(self, certificate: tuple[uuid.UUID, gmpy2.mpz, bytes]) -> bool:
        """
        Проверка сертификата.
        Возвращает признак валидности сертификата.
        """
        
        client_id, client_public_key, signature = certificate
        
        if client_id not in self._db:
            trace('[SchnorrCA]', f"Client {client_id} doesn't exist")
            raise ValueError("Client doesn't exist")
            
        h = SHA256.new(str(client_id).encode() + number.long_to_bytes(int(client_public_key)))
        verifier = pss.new(self.public_key)
        
        try:
            verifier.verify(h, signature)
            return True
        except:
            return False
        
        
    @property
    def public_key(self):
        """
        Получение открытого ключа СА.
        """
        
        return self._public_key
    
    
    @property
    def parameters(self) -> tuple[gmpy2.mpz, gmpy2.mpz, gmpy2.mpz]:
        """
        Параметры протокола (p, q, g)
        """
        
        return self._p, self._q, self._g
        
        
    @staticmethod
    def _generate_parameters() -> tuple[gmpy2.mpz, gmpy2.mpz, gmpy2.mpz]:
        """
        Генерация параметров протокола (p, q, g).
        """
        
        Q_LENGTH = 160
        P_LENGTH = 1024
        
        q = gmpy2.mpz(number.getPrime(Q_LENGTH))
        p = gmpy2.mul(getrandbits(P_LENGTH - Q_LENGTH), q) + 1
        
        while not gmpy2.is_prime(p):
            p = gmpy2.mul(getrandbits(P_LENGTH - Q_LENGTH), q) + 1
            
        #
        # Сгенерирую число, взаимно простое с p
        #
        
        tmp = getrandbits(P_LENGTH)
        while gmpy2.gcd(tmp, p) != 1:
            tmp = getrandbits(P_LENGTH)
        
        #
        # По малой теореме Ферма такой g будет иметь порядок q
        #
        
        g = gmpy2.powmod(tmp, (p - 1) // q, p)
        
        trace('[SchnorrCA]', f'{p = }, {q = }, {g = }')
        return p, q, g

In [15]:
class SchnorrClient(object):
    def __init__(self, parameters: tuple[gmpy2.mpz, gmpy2.mpz, gmpy2.mpz], *,
                 fake_data: tuple[uuid.UUID, gmpy2.mpz, tuple] = None):
        """
        Инициализация клиента. Генерирует идентификатор клиента,
        ключевую пару.
        Последний параметр необходим для демонстрации неуспешной аутентификации.
        """
        
        self._p, self._q, self._g = parameters
        self._id = uuid.uuid4() if fake_data is None else fake_data[0]
        
        self._cert = None if fake_data is None else fake_data[2]
        self._r = None
        
        self._private_key = randint(1, self._q - 1)
        self._public_key  = gmpy2.powmod(
            self._g, -self._private_key, self._p) if fake_data is None else fake_data[1]
        
        trace('[SchnorrClient]', f'Client {self._id}')
        trace('[SchnorrClient]', f'Public key = {self._public_key}')
        
        
    def register(self, ca: SchnorrCA) -> None:
        """
        Регистрация клиента на СА. Принимает дескриптор СА.
        """
        
        self._cert = ca.register_client(self.identifier, self.public_key)
        
        
    def get_x(self) -> gmpy2.mpz:
        """
        Получение числа х, получаемого по формуле:
                    x = g ** r mod p
        где r - случайное целое число из отрезка [1, q - 1].
        Сохраняет значение r.
        """
        
        self._r = randint(1, self._q - 1)
        return gmpy2.powmod(self._g, self._r, self._p)
    
    
    def get_y(self, e, security_parameter) -> gmpy2.mpz:
        """
        Получение числа х, получаемого по формуле:
                    x = a * e + r mod q
        где r - сохраненное в get_x служебное значение, e - число,
        полученное от проверяющего, a - закрытый ключ клиента.
        """
        
        if e > 2 ** security_parameter or e < 1:
            trace('[SchnorrClient]', f'Invalid {e = }')
            raise ValueError('Invalid e')
            
        pre_result = gmpy2.mul(self._private_key, e) + self._r
        return gmpy2.t_mod(pre_result, self._q)
    
    
    @property
    def certificate(self):
        """
        Получение сертификата клиента.
        """
        
        return self._cert
        
        
    @property
    def public_key(self) -> gmpy2.mpz:
        """
        Получение открытого ключа клиента.
        """
        
        return self._public_key
        
        
    @property
    def identifier(self) -> uuid.UUID:
        """
        Получение идентификатора клиента.
        """
        
        return self._id

In [16]:
class SchnorrVerifier(object):
    def __init__(self, parameters, security_parameter):
        """
        Инициализация проверяющего. Фактически только закрепляет параметры протокола.
        """
        
        self._p, self._q, self._g = parameters
        self._security_parameter = security_parameter
        
        assert security_parameter > 0 and 2 ** security_parameter < self._q
        
        self._e_values = set()
        
        
    def authenticate(self, client: SchnorrClient, ca: SchnorrCA) -> bool:
        """
        Аутентификация клиента. На вход принимает дескрипторы клиента и СА.
        Возвращает True в случае успешной аутентификации.
        """
        
        #
        # Получаю с клиента x и его сертификат
        #
        
        x, cert = client.get_x(), client.certificate
        trace('[SchnorrVerifier]', f'{x = }')
        
        #
        # Если сертификат битый или невалидный, то это плохо
        #
        
        if not ca.verify_certificate(cert):
            trace('[SchnorrVerifier]', f'Invalid certificate {cert}')
            return False
            
        #
        # Генерирую число е и отдаю его клиенту
        #
            
        e = randint(1, 2 ** self._security_parameter)
        while e in self._e_values:
            e = randint(1, self._security_parameter * 2)
            
        self._e_values.add(e)        
        trace('[SchnorrVerifier]', f'{e = }')
        
        y = client.get_y(e, 2 ** self._security_parameter)
        trace('[SchnorrVerifier]', f'{y = }')
        
        #
        # Вычисляю значение, которое необходимо сравнить с полученным от клиента х
        # И сразу же сравниваю их
        # 
        
        product = gmpy2.mul(gmpy2.powmod(self._g, y, self._p), 
                            gmpy2.powmod(client.public_key, e, self._p))
        z = gmpy2.t_mod(product, self._p)
        trace('[SchnorrVerifier]', f'{z = }')
        
        return z == x

----

In [17]:
# Создаю доверенный центр
ca = SchnorrCA()

[TRACE] [SchnorrCA] p = mpz(151830363186297732601243730065299544984378916363912122338533264516580695000877364592853668966055600101366443127860218048601009122953269346874611963959050879985223948084259872585427078297369225851835178191164193482492948916715370767997543381556747046365397656645787085760752508152616128797932132220697996204969), q = mpz(1455766506467347948386293587430065436613126777323), g = mpz(131965867237608883301701261064143571579102674156732942296178363965046509946384184900273042739074401039237051992887062284371233510199616535682551797075248047639976561544335792118971456919604392660561197288661417141161490124898399453490833917404455751758318875965513812807110352600170373105046759525154640256873)
[TRACE] [SchnorrCA] RSA public key = (65537, 77013240514127455845929569909197332483589022022369200977303400639180841020942402406875494674714759931482323392947031265746705868449859099933704794341538263316242280953643014760093421689458690061159269566597801194428663203592380308436946

In [18]:
# Создам Алису - валидного пользователя
alice = SchnorrClient(ca.parameters)

[TRACE] [SchnorrClient] Client 43692cbd-0344-45cb-8496-19158c6e4035
[TRACE] [SchnorrClient] Public key = 26833023354038158471251272028178180065713489539219731110144856995680862868575656704062164097647195706987776147474045986146530018600181916494942592685708834038792810964761445329608955173947197547920354402147948279887902084401366377015996093689578796291757322420657137387331967599840090406546204309717735821452


In [19]:
# Регистрирую валидного пользователя
alice.register(ca)

[TRACE] [SchnorrCA] Certificate = (UUID('43692cbd-0344-45cb-8496-19158c6e4035'), mpz(26833023354038158471251272028178180065713489539219731110144856995680862868575656704062164097647195706987776147474045986146530018600181916494942592685708834038792810964761445329608955173947197547920354402147948279887902084401366377015996093689578796291757322420657137387331967599840090406546204309717735821452), b" \xd8\xc3*09\x07qoa\xcd#\xb9\xc0S\x01^\x15-`\xd3\xceZ\x01K\xdfI\x19\xe5\x18\xe0\xaeIuG\x87QvQ\x01\xb1\x1c\\\x16B>\xce\x07\xdd\xe1>\x98\xfdf,\x14t\xde\xea\x07\xdf\xf4\xbc#\x81\xebe\x1fo\xa0\xef\xf6\xb5\x9f&E\x83\xf8\x82=/\x85\\\xd5l\x08\x087\x85\xb8_\x9f\x9e\x14\x06\xa5\xb0j\xfc\x99(\xf2d@\x16o\x9bP\x8c\x96(0y\x95\xc6_o\xdb$\xfe\xc1\xfc\xfc0\xcbK~?V\xabO\x9e\xd9E\xf4\xc6\xce\x7f\x15\xf8\xadX\xb4\x11\xd5%\x8b\xf7:\x809v\xcd\xc8\xc6V\x96\xfdF\x9eR\x8aWt8\xb3\x80-\xb0\xdez\xb7\\\xbdI\xe8\xb8\x0e\xbdx\xd6x\xe84\xd9d\xa7\x92\xf2g\x99\x19\x0e\xc0\xb1rdv\xd3\xa6\xe2?\xfe3H\x9f\xa0PB\x85\xf6\x94}X\x9f\xc

In [20]:
# Параметр безопасности положим равным 10 для демонстрации
security_parameter = 10

# Создаю проверяющую сторону
verifier = SchnorrVerifier(ca.parameters, security_parameter)

In [21]:
# Произвожу аутентификацию валидного пользователя
verifier.authenticate(alice, ca)

[TRACE] [SchnorrVerifier] x = mpz(15700808558092753339003617701852238138790157305635909613046652819913831303306364475259386249528928143612192245218731645801969269380350007645056828732907334650361563596475971513099332473473839740476266894013482157288904645181967618532619582144794164925728888273857943445544135879629043490273488074163741845524)
[TRACE] [SchnorrVerifier] e = mpz(139)
[TRACE] [SchnorrVerifier] y = mpz(173624567712510436470581266894516666272541860971)
[TRACE] [SchnorrVerifier] z = mpz(15700808558092753339003617701852238138790157305635909613046652819913831303306364475259386249528928143612192245218731645801969269380350007645056828732907334650361563596475971513099332473473839740476266894013482157288904645181967618532619582144794164925728888273857943445544135879629043490273488074163741845524)


True

In [22]:
# Создаю Еву - клиента, который пытается представиться Алисой
eve = SchnorrClient(ca.parameters, 
                    fake_data=(alice.identifier, alice.public_key, alice.certificate))

[TRACE] [SchnorrClient] Client 43692cbd-0344-45cb-8496-19158c6e4035
[TRACE] [SchnorrClient] Public key = 26833023354038158471251272028178180065713489539219731110144856995680862868575656704062164097647195706987776147474045986146530018600181916494942592685708834038792810964761445329608955173947197547920354402147948279887902084401366377015996093689578796291757322420657137387331967599840090406546204309717735821452


In [23]:
# Произвожу аутентификацию невалидного пользователя
verifier.authenticate(eve, ca)

[TRACE] [SchnorrVerifier] x = mpz(2999931989798777744807063954668993509297958119707894720187072891822206006868115098504189681259695709482378401953304853035839859532792704973714843993716967950105328153477349630868400209452084086959273766485490126241307565694574495295557544982955815537267159803331199372897885481619754525959455228036308286818)
[TRACE] [SchnorrVerifier] e = mpz(326)
[TRACE] [SchnorrVerifier] y = mpz(1167305336196651232646449590567325630624342057488)
[TRACE] [SchnorrVerifier] z = mpz(5102772393057271040810485677327110916126681161004870633599215498241105508265416457487244287805604519837182633906456120595203090283855027841781057629637264502542871858635163555328508154270409131091234363516643822967695901453939197739132181419795305748318577310584618531081986837822255484258662610247098341350)


False